In [2]:
import pandas as pd
import numpy as np
from six.moves import urllib
import tensorflow as tf
import word2vec
import re
import tempfile

In [5]:
# train word2vec with sample word list dataset
# this might take a minute
vecfile = '/home/claire/Documents/jobs/job-hunting/massively/text8'
word2vec.word2vec(vecfile, '/home/claire/Documents/jobs/job-hunting/massively/text8.bin', size=100, verbose=True)
model = word2vec.load('/home/claire/Documents/jobs/job-hunting/massively/text8.bin')

Starting training using file /home/claire/Documents/jobs/job-hunting/massively/text8


KeyboardInterrupt: 

In [6]:
# load the vector model
model = word2vec.load('/home/claire/Documents/jobs/job-hunting/massively/text8.bin')

In [7]:
# import the sentiment data
df = pd.DataFrame.from_csv('/home/claire/Documents/jobs/job-hunting/massively/sentiment_analysis/dfe_happysad_utf.csv', header=0, sep=',', index_col=0)

In [8]:
# split the data into test and train
df['split'] = np.random.randn(df.shape[0], 1)

msk = np.random.rand(len(df)) <= 0.7

df_train = df[msk]
df_test = df[~msk]

In [9]:
# split training and testing tweets by sadnesss and happiness
sad_train = df_train.loc[df_train['label'] == 'sadness']
happy_train = df_train.loc[df_train['label'] == 'happiness']

sad_test = df_test.loc[df_test['label'] == 'sadness']
happy_test = df_test.loc[df_test['label'] == 'happiness']

In [10]:
# create numpy arrays
np_sad_train = sad_train.as_matrix(columns=df.columns[0:2])
np_happy_train = happy_train.as_matrix(columns=df.columns[0:2])

np_sad_test = sad_test.as_matrix(columns=df.columns[0:2])
np_happy_test = happy_test.as_matrix(columns=df.columns[0:2])

# or create objects for mean embedding vectorizer

In [11]:
# set maximum sentence length and number of dimension for each word vector
maxSeqLength = df_train['features'].str.len().max()
numDimensions = 300 

In [12]:
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

def newline(string):
    newline_pattern = re.compile("[^\n]*\n")
    return re.findall(newline_pattern, string)

In [21]:
class indexedArray(np.ndarray):
    def __new__(cls, *args, **kwargs):
        return np.array(*args, **kwargs).view(indexedArray)
    def index(self, value):
        return np.where(self==value)

In [22]:
firstFile = np.zeros((maxSeqLength), dtype='int32')
vectors = indexedArray(model.vectors)

for tweet in np_sad_train:
    indexCounter = 0
    cleaned = cleanSentences(tweet[0])
    split = cleaned.split()
    print(split)
    for word in split:
        try:
            index = vectors.index(model[word])
            tuples = [x[0] for x in index]
            firstFile[indexCounter] = tuples[0]
        except KeyError:
            firstFile[indexCounter] = 399999 #Vector for unknown words
    indexCounter = indexCounter + 1
firstFile

['funeral', 'ceremonygloomy', 'friday']
['charviray', 'charlene', 'my', 'love', 'i', 'miss', 'you']
['brodyjenner', 'if', 'u', 'watch', 'the', 'hills', 'in', 'london', 'u', 'will', 'realise', 'what', 'tourture', 'it', 'is', 'because', 'were', 'weeks', 'and', 'weeks', 'late', 'i', 'just', 'watch', 'itonlinelol']
['the', 'storm', 'is', 'here', 'and', 'the', 'electricity', 'is', 'gone']
['so', 'sleepy', 'again', 'and', 'its', 'not', 'even', 'that', 'late', 'i', 'fail', 'once', 'again']
['how', 'are', 'you', 'convinced', 'that', 'i', 'have', 'always', 'wanted', 'you', 'what', 'signals', 'did', 'i', 'give', 'offdamn', 'i', 'think', 'i', 'just', 'lost', 'another', 'friend']
['fudge', 'just', 'bsd', 'that', 'whole', 'paper', 'so', 'tired', 'ugh', 'i', 'hate', 'school', 'time', 'to', 'sleep']
['etherradio', 'yeah', 's', 'i', 'feel', 'all', 'funny', 'cause', 'i', 'havent', 'slept', 'enough', 'i', 'woke', 'my', 'mum', 'up', 'cause', 'i', 'was', 'singing', 'shes', 'not', 'impressed', 's', 'you']


KeyboardInterrupt: 

In [ ]:
# use scikit-learn to pipeline actual models that will take tokenised text, 
# vectorize and learn to classify the vectors

In [ ]:
# classify the vectors with a classification model -- SVM, Naive Multinomial Bayesian
# can also do the same thing with weights -- TF-IDF